## IMPORTANTE: Reiniciar Kernel

Se você estiver encontrando erro `OSError: could not get source code`, **reinicie o kernel do notebook** (Ctrl+Shift+P → "Restart Kernel") antes de executar as células abaixo.

## 📦 Setup

In [7]:
# 1. Carregar dados + 
import sys
sys.path.append('..')  # Adiciona o diretório pai
from src.utils.io.io_local import *
from src.utils.io.io_clearml import *

from src.utils.io import load_dataframe
from config import config_custom as config
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_features import executar_pipeline_features
from src.pipelines import treinar_pipeline_completo, treinar_rapido

df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

Dados brutos: (1720, 40)


# Pipelines

## Pipeline processamento

In [8]:
# 1. Carregar dados + # 2. PROCESSAMENTO (Limpeza + Imputação)
df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

# 2. PROCESSAMENTO (Limpeza + Imputação)
df_proc = executar_pipeline_processamento(
    df_raw,
    config_imputacao_customizada=config.CONFIG_IMPUTACAO_CUSTOMIZADA,
    criar_agrupamento_temporal=True,
    nome_coluna_agrupamento='mes-ano'
)
print(f'Após processamento: {df_proc.shape}')
print(f'NAs restantes: {df_proc.isna().sum().sum()}')

Dados brutos: (1720, 40)
🔄 Iniciando pipeline de processamento BASE...
  1️⃣ Aplicando substituições de limpeza...
  2️⃣ Convertendo tipos de dados...
  3️⃣ Imputando valores faltantes...
  4️⃣ Criando agrupamento temporal...
✅ Pipeline BASE concluído! Shape final: (1720, 41)
Após processamento: (1720, 41)
NAs restantes: 0


## Pipeline features

In [9]:
# 3. FEATURES (Codificação + Derivadas + Normalização)
df_feat, artefatos = executar_pipeline_features(
    df_proc,
    # Codificação
    aplicar_codificacao=True,
    metodo_codificacao='label',           # 'label' ou 'onehot'
    sufixo_codificacao='_cod',
    
    # Features derivadas
    criar_features_derivadas=True,
    tipos_features_derivadas=[
        'imc',                            # Índice de Massa Corporal
        'imc_classe',                     # Classe do IMC
        'heat_index',                     # Índice de calor
        'dew_point',                      # Ponto de orvalho
        't*u',                            # Temperatura × Umidade
    ],
    
    # Normalização
    aplicar_normalizacao=True,
    metodo_normalizacao='standard',       # 'standard', 'minmax', 'robust'
    agrupamento_normalizacao='mes-ano',   # Normalizar por grupo
    sufixo_normalizacao='_norm',
)

print(f'Após features: {df_feat.shape}')
print(f'Artefatos criados: {list(artefatos.keys())}')
df_feat.to_csv("../dados/resultados/dados_processados_novas_features.csv")
# Visualizar resultado
df_feat.head()


🎨 Iniciando pipeline de FEATURES...
  1️⃣ Criando features derivadas (5 tipos)...
  2️⃣ Aplicando codificação (label)...
  3️⃣ Aplicando normalização (standard)...
✅ Pipeline FEATURES concluído! Shape final: (1720, 78)
   Novas colunas criadas: 37
Após features: (1720, 78)
Artefatos criados: ['mapeamentos_codificacao', 'colunas_normalizadas', 'artefatos_codificacao', 'artefatos_normalizacao']
  2️⃣ Aplicando codificação (label)...
  3️⃣ Aplicando normalização (standard)...
✅ Pipeline FEATURES concluído! Shape final: (1720, 78)
   Novas colunas criadas: 37
Após features: (1720, 78)
Artefatos criados: ['mapeamentos_codificacao', 'colunas_normalizadas', 'artefatos_codificacao', 'artefatos_normalizacao']


OSError: Cannot save file into a non-existent directory: '..\dados\resultados'

# Pipeline de treinamento

In [ ]:
from config.config_gerais import PARAMS_PADRAO

# Definir coluna alvo e features
coluna_alvo = 'p1'

# Usar apenas features principais para teste
features_treino = [
    'idade', 'sexo_cod', 'peso', 'altura',
    'tmedia', 'ur', 'vel_vento',
]

tipos_modelos ='regressao'

# Filtrar features que existem no DataFrame
features_existentes = [f for f in features_treino if f in df_feat.columns]
print(f"Features para treinamento: {features_existentes}")

# Preparar dados
df_treino = df_feat[features_existentes + [coluna_alvo]].dropna()
print(f"Dataset de treino: {df_treino.shape}")

# 4. TREINAMENTO
print("\n" + "="*60)
print("🤖 INICIANDO PIPELINE DE TREINAMENTO")
print("="*60)

resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema=tipos_modelos,  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)

# Visualizar resultados
print("\n" + "="*60)
print("📊 RESULTADOS DO TREINAMENTO")
print("="*60)

# Nome do melhor modelo
nome_modelo = str(resultado['tabela_comparacao'].index[0])
print(f"\n✓ Melhor modelo: {nome_modelo}")

print(f"\n📈 Métricas principais:")
metricas = resultado['metricas_melhor']
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        print(f"  • {nome}: {valor:.4f}")

print(f"\n💾 Modelo salvo em: {resultado.get('caminho_modelo', 'N/A')}")

print("\n📋 Comparação de modelos (top 5 métricas):")
print(resultado['tabela_comparacao'].head())

In [ ]:
resultados_10_experimentos = {}
for i in range(10):
    print("\n" + "="*60)
    print(f"🤖 INICIANDO EXPERIMENTO {i+1}/10")
    print("="*60)
    
    resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)
    
    resultados_10_experimentos[f'Experimento_{i+1}'] = resultado


In [ ]:
tabelas_comparacao = [dicionario["tabela_comparacao"] for dicionario in resultados_10_experimentos.values()] 
tabelas_comparacao =  [tabela.rename(columns={'Model':'Modelos','Accuracy': 'Acurácia', 'AUC': 'AUC', 'Recall': 'Recall', 'Prec.': 'Prec.', 'F1': 'F1'}) for tabela in tabelas_comparacao]
serie_nomes_modelos = tabelas_comparacao[0]['Modelos']
tabelas_comparacao = [tabela.select_dtypes(include='number') for tabela in tabelas_comparacao] 

In [ ]:

df_desvio_metricas= pd.concat(tabelas_comparacao).groupby(level=0).std()
df_media_metricas = pd.concat(tabelas_comparacao).groupby(level=0).mean()
df_media_desvio_metricas_str = df_media_metricas.round(2).astype(str) + " ± " + df_desvio_metricas.round(2).astype(str)
df_media_desvio_metricas_str = df_media_desvio_metricas_str.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_media_metricas = df_media_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas = df_desvio_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas.to_csv('../dados/resultados/desvio_metricas.csv')
df_media_metricas.to_csv('../dados/resultados/media_metricas.csv')
df_media_desvio_metricas_str.to_csv('../dados/resultados/media_e_desvio_metricas_str.csv')

In [ ]:
df_media_desvio_metricas_str


# ClearML

## Pipeline Completo - Versão Local

**Por que não usar os decorators ClearML aqui?**

Os decorators `@pipeline` e `@component` do ClearML **sempre** tentam criar a infraestrutura de pipeline (tasks, serialização, etc.), mesmo quando configurado para `run_locally=True`. Isso causa erros ao tentar serializar DataFrames grandes e criar tasks automaticamente.

**Solução**: Para execução realmente local (sem servidor ClearML), usamos diretamente as funções originais dos pipelines **SEM** os decorators:
- `executar_pipeline_processamento()` - da pasta src/pipelines
- `treinar_pipeline_completo()` - da pasta src/treinamento

Isso funciona perfeitamente porque são as mesmas funções que você já usa normalmente, apenas sem a camada ClearML em cima.

**Quando usar os pipelines com decorators ClearML?**
- Quando você tem um servidor ClearML configurado
- Quando quer enviar jobs para execução remota
- Quando precisa de rastreamento completo no servidor ClearML

In [ ]:
# =========================================================================
# PIPELINE COMPLETO COM CLEARML - REGISTRO TOTAL
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, OutputModel
import pandas as pd
from pathlib import Path

# Imports dos pipelines
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_treinamento_unified import treinar_pipeline_completo

# Configuração
PROJECT_NAME = "conforto_termico"
coluna_alvo = 'p1'

print("="*80)
print("PIPELINE COMPLETO COM CLEARML - FULL TRACKING")
print("="*80)

# ============================================================================
# ETAPA 1: UPLOAD DE DADOS BRUTOS
# ============================================================================
print("\n[ETAPA 1] Upload de Dados Brutos para ClearML")
print("-"*80)

# Criar dataset de dados brutos
dataset_bruto = Dataset.create(
    dataset_name="dados_brutos_conforto_termico",
    dataset_project=PROJECT_NAME,
    description="Dados brutos de conforto termico de Santa Maria"
)

# Salvar temporariamente para upload
temp_path = Path("../dados/temp_clearml")
temp_path.mkdir(exist_ok=True)
arquivo_bruto = temp_path / "dados_brutos.csv"
df_raw.to_csv(arquivo_bruto, index=False)

# Adicionar arquivo ao dataset
dataset_bruto.add_files(str(arquivo_bruto))
dataset_bruto.upload()
dataset_bruto.finalize()

print(f"Dataset bruto criado: ID = {dataset_bruto.id}")
print(f"  - Shape: {df_raw.shape}")
print(f"  - Colunas: {len(df_raw.columns)}")

# ============================================================================
# ETAPA 2: PIPELINE DE PROCESSAMENTO
# ============================================================================
print("\n[ETAPA 2] Pipeline de Processamento")
print("-"*80)

# Criar task de processamento (COM FIX PARA NOTEBOOKS)
task_processamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Processamento",
    task_type=Task.TaskTypes.data_processing,
    reuse_last_task_id=False,  # Não reusar task anterior
    continue_last_task=False,   # Criar nova task sempre
    auto_connect_frameworks=False,  # Desabilita captura automática
    auto_resource_monitoring=False  # Desabilita monitoramento de recursos
)

# Conectar dataset de entrada
task_processamento.connect_configuration({
    "dataset_id": dataset_bruto.id,
    "dataset_name": "dados_brutos_conforto_termico"
})

# Executar processamento
print("Executando processamento...")
df_processado = executar_pipeline_processamento(df_raw)

# Registrar métricas de processamento
task_processamento.get_logger().report_single_value("linhas_entrada", df_raw.shape[0])
task_processamento.get_logger().report_single_value("linhas_saida", df_processado.shape[0])
task_processamento.get_logger().report_single_value("colunas_entrada", df_raw.shape[1])
task_processamento.get_logger().report_single_value("colunas_saida", df_processado.shape[1])
task_processamento.get_logger().report_single_value("nas_removidos", df_raw.isna().sum().sum() - df_processado.isna().sum().sum())

print(f"Dados processados: {df_processado.shape}")
print(f"NAs restantes: {df_processado.isna().sum().sum()}")

# Upload de dados processados como dataset
dataset_processado = Dataset.create(
    dataset_name="dados_processados_conforto_termico",
    dataset_project=PROJECT_NAME,
    parent_datasets=[dataset_bruto.id],
    description="Dados processados (sem NAs, limpeza aplicada)"
)

arquivo_processado = temp_path / "dados_processados.csv"
df_processado.to_csv(arquivo_processado, index=False)
dataset_processado.add_files(str(arquivo_processado))
dataset_processado.upload()
dataset_processado.finalize()

print(f"Dataset processado criado: ID = {dataset_processado.id}")

# Registrar como artefato na task
task_processamento.upload_artifact(
    "dados_processados_sample",
    artifact_object=df_processado.head(100)  # Apenas amostra
)

task_processamento.close()
print("Task de processamento finalizada!")

# ============================================================================
# ETAPA 3: PIPELINE DE TREINAMENTO
# ============================================================================
print("\n[ETAPA 3] Pipeline de Treinamento")
print("-"*80)

# Criar task de treinamento (COM FIX PARA NOTEBOOKS)
task_treinamento = Task.init(
    project_name=PROJECT_NAME,
    task_name="Pipeline_Treinamento",
    task_type=Task.TaskTypes.training,
    reuse_last_task_id=False,
    continue_last_task=False,
    auto_connect_frameworks=False,
    auto_resource_monitoring=False
)

# Conectar dataset processado
task_treinamento.connect_configuration({
    "dataset_id": dataset_processado.id,
    "coluna_alvo": coluna_alvo,
    "tipo_problema": "regressao"
})

# Preparar dados
df_treino = df_processado.dropna()
print(f"Dados para treino: {df_treino.shape}")

# Executar treinamento
print("Executando treinamento...")
resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',
    n_modelos_comparar=3,
    otimizar_hiperparametros=True,
    n_iter_otimizacao=10,
    salvar_modelo_final=True,
    nome_modelo="modelo_conforto_termico_clearml",
    pasta_modelos="../modelos"
)

# ============================================================================
# ETAPA 4: REGISTRO DE MÉTRICAS
# ============================================================================
print("\n[ETAPA 4] Registro de Métricas")
print("-"*80)

logger = task_treinamento.get_logger()

# Métricas do melhor modelo
metricas = resultado['metricas_melhor']
print("\nMetricas registradas:")
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        logger.report_single_value(nome, valor)
        print(f"  {nome}: {valor:.4f}")

# ============================================================================
# ETAPA 5: UPLOAD DE TABELAS E ARTEFATOS
# ============================================================================
print("\n[ETAPA 5] Upload de Tabelas e Artefatos")
print("-"*80)

# Tabela de comparação de modelos
tabela_comparacao = resultado['tabela_comparacao']
task_treinamento.upload_artifact(
    "comparacao_modelos",
    artifact_object=tabela_comparacao
)
print("Tabela de comparacao enviada como artefato")

# Criar gráfico de comparação de modelos
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))
tabela_comparacao[['MAE', 'MSE', 'RMSE', 'R2']].plot(kind='bar', ax=ax)
ax.set_title('Comparacao de Modelos - Metricas')
ax.set_xlabel('Modelos')
ax.set_ylabel('Valor')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# Enviar gráfico
logger.report_matplotlib_figure(
    title="Comparacao de Modelos",
    series="Metricas",
    figure=fig,
    iteration=0
)
plt.close()
print("Grafico de comparacao enviado")

# ============================================================================
# ETAPA 6: REGISTRO DO MODELO
# ============================================================================
print("\n[ETAPA 6] Registro do Modelo")
print("-"*80)

# Criar OutputModel
melhor_modelo_nome = str(resultado['tabela_comparacao'].index[0])
output_model = OutputModel(
    task=task_treinamento,
    name=f"modelo_{melhor_modelo_nome}_conforto_termico",
    framework="PyCaret"
)

# Registrar informações do modelo
output_model.update_labels({
    "tipo": "regressao",
    "coluna_alvo": coluna_alvo,
    "melhor_modelo": melhor_modelo_nome
})

# Adicionar métricas ao modelo
output_model.update_design(config_dict={
    "metricas": {k: float(v) for k, v in metricas.items() if isinstance(v, (int, float))},
    "n_features": df_treino.shape[1] - 1,
    "n_samples_treino": df_treino.shape[0]
})

# Upload do arquivo do modelo
if 'caminho_modelo' in resultado and resultado['caminho_modelo']:
    caminho_modelo = Path(resultado['caminho_modelo'])
    if caminho_modelo.exists():
        output_model.update_weights(weights_filename=str(caminho_modelo))
        print(f"Modelo registrado: {caminho_modelo}")
    else:
        print(f"AVISO: Arquivo do modelo nao encontrado: {caminho_modelo}")

# ============================================================================
# ETAPA 7: FINALIZAÇÃO E RESUMO
# ============================================================================
print("\n[ETAPA 7] Finalizacao")
print("-"*80)

# Tags na task
task_treinamento.add_tags([
    "producao",
    melhor_modelo_nome,
    f"r2_{metricas.get('R2', 0):.2f}".replace(".", "_")
])

task_treinamento.close()

# Limpar arquivos temporários
import shutil
if temp_path.exists():
    shutil.rmtree(temp_path)

# ============================================================================
# RESUMO FINAL
# ============================================================================
print("\n" + "="*80)
print("PIPELINE COMPLETO FINALIZADO - TUDO REGISTRADO NO CLEARML!")
print("="*80)
print(f"\nProjeto: {PROJECT_NAME}")
print(f"\nDatasets criados:")
print(f"  1. Dados brutos: {dataset_bruto.id}")
print(f"  2. Dados processados: {dataset_processado.id}")
print(f"\nTasks criadas:")
print(f"  1. Processamento: {task_processamento.id}")
print(f"  2. Treinamento: {task_treinamento.id}")
print(f"\nModelo registrado:")
print(f"  - Nome: modelo_{melhor_modelo_nome}_conforto_termico")
print(f"  - Melhor modelo: {melhor_modelo_nome}")
print(f"  - R2: {metricas.get('R2', 0):.4f}")
print(f"  - MAE: {metricas.get('MAE', 0):.4f}")
print("\nAcesse o ClearML UI para visualizar todos os resultados!")
print("="*80)

## Visualizar Resultados no ClearML

Após executar a célula acima, acesse a interface web do ClearML:
- **Local**: http://localhost:8080
- **Demo**: https://app.clear.ml

### O que você vai encontrar:

**📊 No Projeto "conforto_termico":**
- 2 Tasks (Processamento + Treinamento)
- 2 Datasets (Bruto + Processado) com genealogia
- 1 Modelo registrado com métricas

**📈 Métricas rastreadas:**
- Todas as métricas de regressão (MAE, MSE, RMSE, R2, etc.)
- Comparação entre modelos
- Gráficos de comparação

**📦 Artefatos salvos:**
- Tabela de comparação de modelos
- Sample dos dados processados
- Gráfico de métricas

**🤖 Modelo:**
- Arquivo .pkl do modelo treinado
- Labels e configurações
- Genealogia completa (dados → processamento → treinamento)

## [OPCIONAL] Buscar e Reusar Recursos do ClearML

In [ ]:
# =========================================================================
# REUSAR RECURSOS DO CLEARML (Datasets, Modelos, Tasks)
# =========================================================================
import sys
sys.path.append('..')

from clearml import Task, Dataset, Model
import pandas as pd

PROJECT_NAME = "conforto_termico_def"

print("="*80)
print("BUSCANDO RECURSOS NO CLEARML")
print("="*80)

# ============================================================================
# 1. LISTAR DATASETS DISPONÍVEIS
# ============================================================================
print("\n[1] Datasets disponiveis no projeto:")
print("-"*80)

datasets = Dataset.list_datasets(
    dataset_project=PROJECT_NAME,
    partial_name="conforto_termico"
)

for i, ds in enumerate(datasets, 1):
    print(f"\n{i}. {ds.name}")
    print(f"   ID: {ds.id}")
    print(f"   Versao: {ds.version}")
    print(f"   Criado: {ds.created}")

# ============================================================================
# 2. BAIXAR DATASET ESPECÍFICO
# ============================================================================
print("\n[2] Baixando dataset processado:")
print("-"*80)

# Buscar o dataset mais recente de dados processados
dataset_processado = Dataset.get(
    dataset_project=PROJECT_NAME,
    dataset_name="dados_processados_conforto_termico"
)

# Baixar para pasta local
local_path = dataset_processado.get_local_copy()
print(f"Dataset baixado para: {local_path}")

# Carregar dados
arquivos = list(Path(local_path).glob("*.csv"))
if arquivos:
    df_downloaded = pd.read_csv(arquivos[0])
    print(f"Dados carregados: {df_downloaded.shape}")
    print(df_downloaded.head())

# ============================================================================
# 3. LISTAR TASKS DO PROJETO
# ============================================================================
print("\n[3] Tasks do projeto:")
print("-"*80)

tasks = Task.get_tasks(
    project_name=PROJECT_NAME,
    task_name=None  # Todas as tasks
)

for i, t in enumerate(tasks[:5], 1):  # Primeiras 5
    print(f"\n{i}. {t.name}")
    print(f"   ID: {t.id}")
    print(f"   Status: {t.status}")
    print(f"   Tipo: {t.task_type}")

# ============================================================================
# 4. BUSCAR MODELOS REGISTRADOS
# ============================================================================
print("\n[4] Modelos registrados:")
print("-"*80)

modelos = Model.query_models(
    project_name=PROJECT_NAME,
    model_name=None  # Todos os modelos
)

for i, modelo in enumerate(modelos, 1):
    print(f"\n{i}. {modelo.name}")
    print(f"   ID: {modelo.id}")
    print(f"   Framework: {modelo.framework}")
    print(f"   Labels: {modelo.labels}")
    
    # Baixar modelo se quiser
    # modelo_local = modelo.get_local_copy()
    # print(f"   Download: {modelo_local}")

# ============================================================================
# 5. CLONAR E REUSAR TASK
# ============================================================================
print("\n[5] Exemplo: Clonar uma task existente")
print("-"*80)

if tasks:
    task_original = tasks[0]
    print(f"Clonando task: {task_original.name}")
    
    # Clonar task (cria uma cópia)
    task_clonada = Task.clone(
        source_task=task_original.id,
        name=f"{task_original.name}_clonada",
        project=PROJECT_NAME
    )
    
    print(f"Task clonada criada: {task_clonada.id}")
    print("Você pode executar esta task clonada com configuracoes diferentes!")

print("\n" + "="*80)
print("BUSCA DE RECURSOS CONCLUIDA")
print("="*80)

In [1]:
# PASSO 1: Verificar Importações e Credenciais
# ============================================================================

import sys
from pathlib import Path

# Adicionar path do projeto
if '..' not in sys.path:
    sys.path.append('..')

print("="*80)
print("DIAGNÓSTICO COMPLETO")
print("="*80)

# 1. Verificar arquivo .env
print("\n[1] Verificando arquivo .env...")
env_path = Path('../.env')
if env_path.exists():
    print(f"✓ Arquivo .env encontrado: {env_path.resolve()}")
    # Ler conteúdo (sem mostrar secrets)
    with open(env_path, 'r') as f:
        linhas = f.readlines()
    print(f"  Linhas no arquivo: {len(linhas)}")
    for linha in linhas:
        if '=' in linha and not linha.startswith('#'):
            chave = linha.split('=')[0].strip()
            print(f"  • {chave}: {'***' if 'KEY' in chave else '✓'}")
else:
    print(f"✗ Arquivo .env NÃO encontrado em: {env_path.resolve()}")

# 2. Testar carregamento de credenciais
print("\n[2] Testando carregamento de credenciais...")
try:
    from src.clearml.utils.credenciais_clearml import configurar_clearml_online
    sucesso = configurar_clearml_online()
    if sucesso:
        print("✓ Credenciais carregadas com sucesso")
    else:
        print("✗ Falha ao carregar credenciais")
except Exception as e:
    print(f"✗ Erro ao importar/configurar: {e}")
    import traceback
    traceback.print_exc()

# 3. Verificar importação do pipeline
print("\n[3] Testando importação do pipeline...")
try:
    from src.clearml.pipelines_clearml import executar_pipeline_processamento_clearml
    print("✓ Pipeline importado com sucesso")
except Exception as e:
    print(f"✗ Erro ao importar pipeline: {e}")
    import traceback
    traceback.print_exc()

# 4. Verificar arquivo de dados
print("\n[4] Verificando arquivo de dados...")
arquivo_dados = Path('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
if arquivo_dados.exists():
    print(f"✓ Arquivo encontrado: {arquivo_dados.resolve()}")
    print(f"  Tamanho: {arquivo_dados.stat().st_size / (1024*1024):.2f} MB")
else:
    print(f"✗ Arquivo NÃO encontrado: {arquivo_dados.resolve()}")

print("\n" + "="*80)
print("Se tudo estiver ✓, execute a próxima célula para rodar o pipeline")
print("="*80)

DIAGNÓSTICO COMPLETO

[1] Verificando arquivo .env...
✓ Arquivo .env encontrado: C:\Users\Readone\Desktop\tcc_mba_esalq_mlops\.env
  Linhas no arquivo: 5
  • CLEARML_WEB_HOST: ✓
  • CLEARML_API_HOST: ✓
  • CLEARML_FILES_HOST: ✓
  • CLEARML_API_ACCESS_KEY: ***
  • CLEARML_API_SECRET_KEY: ***

[2] Testando carregamento de credenciais...


2026-02-09 22:39:31,707 - INFO - Módulo ClearML carregado com sucesso
2026-02-09 22:39:31,714 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:39:31,715 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:39:31,715 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:39:31,716 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-09 22:39:31,714 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:39:31,715 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:39:31,715 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:39:31,716 - INFO - ✓ ClearML configurado para uso ONLINE


✓ Credenciais carregadas com sucesso

[3] Testando importação do pipeline...
✓ Pipeline importado com sucesso

[4] Verificando arquivo de dados...
✓ Arquivo encontrado: C:\Users\Readone\Desktop\tcc_mba_esalq_mlops\dados\2025.05.14_thermal_confort_santa_maria_brazil_.csv
  Tamanho: 0.29 MB

Se tudo estiver ✓, execute a próxima célula para rodar o pipeline


In [2]:
# Processamento
# ============================================================================
import traceback
from src.clearml.pipelines_clearml import executar_pipeline_processamento_clearml

arquivo = '../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv'

print("="*80)
print("PIPELINE DE PROCESSAMENTO - VERSÃO FINAL")
print("="*80)
print(f"Arquivo: {arquivo}")
print("Modo: ONLINE (ClearML)")
print("\nMelhorias:")
print("  ✓ Carregamento robusto com load_dataframe() de src/utils/io")
print("  ✓ Detecção automática de delimitador")
print("  ✓ Métricas de entrada/saída registradas")
print("="*80 + "\n")

try:
    resultado = executar_pipeline_processamento_clearml(
        caminho_csv=arquivo,
        offline_mode=False
    )
    
    print("\n" + "="*80)
    print("✅ PIPELINE CONCLUÍDO COM SUCESSO!")
    print("="*80)
    print(f"\n📊 Resultados:")
    print(f"  Shape final: {resultado['shape']}")
    print(f"  Dataset ID: {resultado.get('dataset_id', 'N/A')}")
    print(f"  NAs removidos: {resultado.get('nas_removidos', 'N/A')}")
    
    print(f"\n🌐 Acesse: https://app.clear.ml")
    print(f"  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS")
    print(f"  • Task: Pipeline_Processamento")
    print(f"  • Dataset: dados_processados (ID: {resultado.get('dataset_id', 'N/A')[:8]}...)")
    print(f"  • Métricas: linhas entrada={resultado['shape'][0]}, NAs removidos={resultado.get('nas_removidos', 'N/A')}")
    
    print("\n💾 DataFrame processado disponível em:")
    print("  resultado['df_processado']")
    print("="*80)
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ ERRO DETECTADO")
    print("="*80)
    print(f"\nTipo do erro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    print("\n" + "-"*80)
    print("TRACEBACK COMPLETO:")
    print("-"*80)
    traceback.print_exc()


2026-02-09 22:39:31,753 - INFO - Modo ONLINE: Iniciando integração ClearML
2026-02-09 22:39:31,755 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:39:31,756 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:39:31,757 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:39:31,758 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-09 22:39:31,755 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:39:31,756 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:39:31,757 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:39:31,758 - INFO - ✓ ClearML configurado para uso ONLINE


PIPELINE DE PROCESSAMENTO - VERSÃO FINAL
Arquivo: ../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv
Modo: ONLINE (ClearML)

Melhorias:
  ✓ Carregamento robusto com load_dataframe() de src/utils/io
  ✓ Detecção automática de delimitador
  ✓ Métricas de entrada/saída registradas

ClearML Task: created new task id=a4ae6d82b61443279bff4087ad551ed1
ClearML Task: created new task id=a4ae6d82b61443279bff4087ad551ed1
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/a4ae6d82b61443279bff4087ad551ed1/output/log
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/a4ae6d82b61443279bff4087ad551ed1/output/log


2026-02-09 22:39:42,955 - INFO - Task criada: Pipeline_Processamento (ID: a4ae6d82b61443279bff4087ad551ed1, Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS)
2026-02-09 22:39:46,740 - INFO - ✓ Task ClearML criada (ID: a4ae6d82b61443279bff4087ad551ed1)
2026-02-09 22:39:46,743 - INFO - ================================================================================
2026-02-09 22:39:46,745 - INFO - PIPELINE DE PROCESSAMENTO
2026-02-09 22:39:46,745 - INFO - ================================================================================
2026-02-09 22:39:46,746 - INFO - 
[1] Carregando dados do CSV...
2026-02-09 22:39:46,746 - INFO -     Arquivo: ../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv
2026-02-09 22:39:46,740 - INFO - ✓ Task ClearML criada (ID: a4ae6d82b61443279bff4087ad551ed1)
2026-02-09 22:39:46,743 - INFO - ================================================================================
2026-02-09 22:39:46,745 - INFO - PIPELINE DE PROCESSAMENTO
2026-02-0

🔄 Iniciando pipeline de processamento BASE...
  1️⃣ Aplicando substituições de limpeza...
  2️⃣ Convertendo tipos de dados...
  3️⃣ Imputando valores faltantes...
  4️⃣ Criando agrupamento temporal...
✅ Pipeline BASE concluído! Shape final: (1720, 41)


2026-02-09 22:39:49,995 - INFO - DataFrame registrado: dados_processados_sample (shape: (100, 41))
2026-02-09 22:39:49,996 - INFO - ✓ Artefatos registrados
2026-02-09 22:39:49,997 - INFO - 
[4] Criando dataset ClearML...
2026-02-09 22:39:49,996 - INFO - ✓ Artefatos registrados
2026-02-09 22:39:49,997 - INFO - 
[4] Criando dataset ClearML...


ClearML results page: https://app.clear.ml/projects/801c2956f7704234b987beef3f9cc1e6/experiments/eed3f4f2f711470882340222902528e9/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/801c2956f7704234b987beef3f9cc1e6/experiments/eed3f4f2f711470882340222902528e9


2026-02-09 22:40:14,877 - INFO - Dataset criado: dados_processados (ID: eed3f4f2f711470882340222902528e9, Projeto: Datasets)


Uploading dataset changes (1 files compressed to 49.62 KiB) to https://files.clear.ml
File compression and upload completed: total size 49.62 KiB, 1 chunk(s) stored (average size 49.62 KiB)
File compression and upload completed: total size 49.62 KiB, 1 chunk(s) stored (average size 49.62 KiB)


2026-02-09 22:40:42,137 - INFO - ✓ Dataset criado (ID: eed3f4f2f711470882340222902528e9)
2026-02-09 22:40:42,139 - INFO - 
[5] Fechando task...
2026-02-09 22:40:42,139 - INFO - 
[5] Fechando task...
2026-02-09 22:40:45,295 - INFO - ✓ Task finalizada e pronta para próximo pipeline
2026-02-09 22:40:45,295 - INFO - 
2026-02-09 22:40:45,296 - INFO - PIPELINE CONCLUÍDO COM SUCESSO
2026-02-09 22:40:45,297 - INFO - ================================================================================
2026-02-09 22:40:45,295 - INFO - ✓ Task finalizada e pronta para próximo pipeline
2026-02-09 22:40:45,295 - INFO - 
2026-02-09 22:40:45,296 - INFO - PIPELINE CONCLUÍDO COM SUCESSO
2026-02-09 22:40:45,297 - INFO - ================================================================================



✅ PIPELINE CONCLUÍDO COM SUCESSO!

📊 Resultados:
  Shape final: (1720, 41)
  Dataset ID: eed3f4f2f711470882340222902528e9
  NAs removidos: 435

🌐 Acesse: https://app.clear.ml
  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS
  • Task: Pipeline_Processamento
  • Dataset: dados_processados (ID: eed3f4f2...)
  • Métricas: linhas entrada=1720, NAs removidos=435

💾 DataFrame processado disponível em:
  resultado['df_processado']


In [3]:
# ============================================================================
# PIPELINE 2: ENGENHARIA DE FEATURES COM CLEARML
# ============================================================================
# Recebe o resultado do Pipeline 1 e aplica transformações de features
# ============================================================================

import traceback
from src.clearml.pipelines_clearml import executar_pipeline_features_clearml

print("="*80)
print("PIPELINE DE FEATURES - CLEARML ONLINE")
print("="*80)

# Verificar se temos o resultado do pipeline anterior
if 'resultado' not in locals():
    print("\n⚠️  ATENÇÃO: Execute primeiro o Pipeline 1 (Processamento)")
    print("Usando fallback: carregando dados do notebook...")
    
    # Fallback: usar df_raw do notebook
    from src.pipelines.pipeline_processamento import executar_pipeline_processamento
    df_para_features = executar_pipeline_processamento(df_raw)
    dataset_anterior_id = None
else:
    print(f"✓ Pipeline 1 concluído")
    print(f"  Dataset ID anterior: {resultado.get('dataset_id', 'N/A')}")
    df_para_features = resultado['dados_processados']
    dataset_anterior_id = resultado.get('dataset_id')

print(f"\nDados para features: {df_para_features.shape}")
print("="*80 + "\n")

try:
    resultado_features = executar_pipeline_features_clearml(
        df_processado=df_para_features,
        dataset_processado_id=dataset_anterior_id,
        offline_mode=False,
        # Configurações de features
        aplicar_codificacao=True,
        metodo_codificacao='label',
        criar_features_derivadas=True,
        tipos_features_derivadas=['imc', 'imc_classe', 'heat_index', 'dew_point', 't*u'],
        aplicar_normalizacao=True,
        metodo_normalizacao='standard'
    )
    
    print("\n" + "="*80)
    print("✅ PIPELINE DE FEATURES CONCLUÍDO!")
    print("="*80)
    print(f"\n📊 Resultados:")
    print(f"  Shape final: {resultado_features['shape']}")
    
    # Formatação segura do Dataset ID
    dataset_id = resultado_features.get('dataset_id', 'N/A')
    if dataset_id and dataset_id != 'N/A':
        dataset_id_display = f"{dataset_id[:8]}..."
    else:
        dataset_id_display = "N/A (modo offline ou erro na criação)"
    print(f"  Dataset ID: {dataset_id_display}")
    
    print(f"  Artefatos criados: {len(resultado_features.get('artefatos', {}))}")
    
    # Mostrar artefatos criados
    if resultado_features.get('artefatos'):
        print(f"\n📦 Artefatos:")
        for nome, artefato in resultado_features['artefatos'].items():
            if isinstance(artefato, dict) and 'shape' in artefato:
                print(f"  • {nome}: {artefato['shape']}")
            else:
                print(f"  • {nome}")
    
    print(f"\n🌐 Acesse: https://app.clear.ml")
    print(f"  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS")
    print(f"  • Task: Pipeline_Features")
    print(f"  • Dataset: dados_features (ID: {dataset_id_display})")
    print(f"  • Genealogia: dados_brutos → processados → features")
    
    print("\n💾 DataFrame com features disponível em:")
    print("  resultado_features['dados_features']")
    
    # Estatísticas das features criadas
    if 'artefatos' in resultado_features:
        print(f"\n📈 Transformações aplicadas:")
        if 'mapeamentos_codificacao' in resultado_features['artefatos']:
            print(f"  ✓ Codificação: {len(resultado_features['artefatos']['mapeamentos_codificacao'])} variáveis")
        if 'colunas_normalizadas' in resultado_features['artefatos']:
            print(f"  ✓ Normalização: {len(resultado_features['artefatos']['colunas_normalizadas'])} colunas")
    
    print("="*80)
    
except Exception as e:
    print("\n" + "="*80)
    print("❌ ERRO DETECTADO")
    print("="*80)
    print(f"\nTipo do erro: {type(e).__name__}")
    print(f"Mensagem: {str(e)}")
    print("\n" + "-"*80)
    print("TRACEBACK COMPLETO:")
    print("-"*80)
    traceback.print_exc()
    print("\n" + "="*80)
    print("💡 Dicas:")
    print("  • Execute primeiro o Pipeline 1 (Processamento)")
    print("  • Verifique se o DataFrame está no formato correto")
    print("  • Use offline_mode=True para executar sem ClearML")
    print("="*80)

2026-02-09 22:40:45,321 - INFO - Modo ONLINE: Iniciando integração ClearML
2026-02-09 22:40:45,323 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:40:45,323 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:40:45,324 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:40:45,325 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-09 22:40:45,323 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:40:45,323 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:40:45,324 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:40:45,325 - INFO - ✓ ClearML configurado para uso ONLINE


PIPELINE DE FEATURES - CLEARML ONLINE
✓ Pipeline 1 concluído
  Dataset ID anterior: eed3f4f2f711470882340222902528e9

Dados para features: (1720, 41)

ClearML Task: created new task id=3591669706a0414a90dd7cd5538de23a
ClearML Task: created new task id=3591669706a0414a90dd7cd5538de23a
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/3591669706a0414a90dd7cd5538de23a/output/log
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/3591669706a0414a90dd7cd5538de23a/output/log


2026-02-09 22:40:54,872 - INFO - Task criada: Pipeline_Features (ID: 3591669706a0414a90dd7cd5538de23a, Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS)
2026-02-09 22:40:58,522 - INFO - ✓ Task ClearML criada (ID: 3591669706a0414a90dd7cd5538de23a)
2026-02-09 22:40:58,523 - INFO - ================================================================================
2026-02-09 22:40:58,524 - INFO - PIPELINE DE ENGENHARIA DE FEATURES
2026-02-09 22:40:58,526 - INFO - ================================================================================
2026-02-09 22:40:58,527 - INFO - 
[1] Executando pipeline de features local...
2026-02-09 22:40:58,527 - INFO -     Shape entrada: (1720, 41)
2026-02-09 22:40:58,522 - INFO - ✓ Task ClearML criada (ID: 3591669706a0414a90dd7cd5538de23a)
2026-02-09 22:40:58,523 - INFO - ================================================================================
2026-02-09 22:40:58,524 - INFO - PIPELINE DE ENGENHARIA DE FEATURES
2026-02-09 22:40:58,526 -

🎨 Iniciando pipeline de FEATURES...
  1️⃣ Criando features derivadas (5 tipos)...
  2️⃣ Aplicando codificação (label)...
  3️⃣ Aplicando normalização (standard)...
✅ Pipeline FEATURES concluído! Shape final: (1720, 78)
   Novas colunas criadas: 37


2026-02-09 22:41:01,883 - INFO - DataFrame registrado: dados_features (shape: (1720, 78))
2026-02-09 22:41:03,446 - INFO - Arquivo registrado: artefatos_features
2026-02-09 22:41:03,448 - INFO -   Artefatos registrados: ['mapeamentos_codificacao', 'colunas_normalizadas', 'artefatos_codificacao', 'artefatos_normalizacao']
2026-02-09 22:41:03,449 - INFO - Métricas registradas: ['linhas_features', 'colunas_features', 'novas_colunas', 'colunas_entrada']
2026-02-09 22:41:03,450 - INFO - ✓ Artefatos registrados
2026-02-09 22:41:03,451 - INFO - 
[3] Criando dataset ClearML...
2026-02-09 22:41:03,446 - INFO - Arquivo registrado: artefatos_features
2026-02-09 22:41:03,448 - INFO -   Artefatos registrados: ['mapeamentos_codificacao', 'colunas_normalizadas', 'artefatos_codificacao', 'artefatos_normalizacao']
2026-02-09 22:41:03,449 - INFO - Métricas registradas: ['linhas_features', 'colunas_features', 'novas_colunas', 'colunas_entrada']
2026-02-09 22:41:03,450 - INFO - ✓ Artefatos registrados
202

...

ClearML results page: https://app.clear.ml/projects/d002871436ec4f3a8e158a0997a0e923/experiments/80272a4490934b8ba191125924fc6c34/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/d002871436ec4f3a8e158a0997a0e923/experiments/80272a4490934b8ba191125924fc6c34


2026-02-09 22:41:53,903 - INFO - Dataset criado: dados_features (ID: 80272a4490934b8ba191125924fc6c34, Projeto: Datasets)


Uploading dataset changes (1 files compressed to 137.64 KiB) to https://files.clear.ml
File compression and upload completed: total size 137.64 KiB, 1 chunk(s) stored (average size 137.64 KiB)
File compression and upload completed: total size 137.64 KiB, 1 chunk(s) stored (average size 137.64 KiB)


2026-02-09 22:43:28,236 - INFO - ✓ Dataset criado (ID: 80272a4490934b8ba191125924fc6c34)
2026-02-09 22:43:28,237 - INFO - 
2026-02-09 22:43:28,238 - INFO - PIPELINE DE FEATURES CONCLUÍDO COM SUCESSO
2026-02-09 22:43:28,239 - INFO - ================================================================================
2026-02-09 22:43:28,237 - INFO - 
2026-02-09 22:43:28,238 - INFO - PIPELINE DE FEATURES CONCLUÍDO COM SUCESSO
2026-02-09 22:43:28,239 - INFO - ================================================================================



✅ PIPELINE DE FEATURES CONCLUÍDO!

📊 Resultados:
  Shape final: (1720, 78)
  Dataset ID: 80272a44...
  Artefatos criados: 4

📦 Artefatos:
  • mapeamentos_codificacao
  • colunas_normalizadas
  • artefatos_codificacao
  • artefatos_normalizacao

🌐 Acesse: https://app.clear.ml
  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS
  • Task: Pipeline_Features
  • Dataset: dados_features (ID: 80272a44...)
  • Genealogia: dados_brutos → processados → features

💾 DataFrame com features disponível em:
  resultado_features['dados_features']

📈 Transformações aplicadas:
  ✓ Codificação: 2 variáveis
  ✓ Normalização: 30 colunas


In [ ]:
# ============================================================================
# PIPELINE 3: TREINAMENTO DE MODELOS COM CLEARML
# ============================================================================
# Recebe o resultado do Pipeline 2 e treina modelos com PyCaret
# ============================================================================

import traceback
from src.clearml.pipelines_clearml import executar_pipeline_treinamento_clearml

print("="*80)
print("PIPELINE DE TREINAMENTO - CLEARML ONLINE")
print("="*80)

# Verificar se temos o resultado do pipeline anterior
if 'resultado_features' not in locals():
    print("\n⚠️  ATENÇÃO: Execute primeiro os Pipelines 1 e 2")
    print("Usando fallback: tentando usar dados do notebook...")
    
    # Fallback: verificar se temos df_feat do notebook
    if 'df_feat' in locals():
        print("✓ Usando df_feat do notebook")
        df_para_treino = df_feat
        dataset_anterior_id = None
    else:
        print("❌ Nenhum dado disponível. Execute os pipelines anteriores primeiro!")
        df_para_treino = None
        dataset_anterior_id = None
else:
    print(f"✓ Pipeline 2 concluído")
    print(f"  Dataset ID anterior: {resultado_features.get('dataset_id', 'N/A')}")
    df_para_treino = resultado_features['dados_features']
    dataset_anterior_id = resultado_features.get('dataset_id')

if df_para_treino is not None:
    # Preparar dados (remover NAs)
    df_treino_final = df_para_treino.dropna()
    
    print(f"\nDados para treinamento: {df_treino_final.shape}")
    print(f"Coluna alvo: p1")
    print("="*80 + "\n")

    try:
        resultado_treinamento = executar_pipeline_treinamento_clearml(
            df_features=df_treino_final,
            coluna_alvo='p1',
            tipo_problema='regressao',
            dataset_features_id=dataset_anterior_id,
            offline_mode=False,
            # Configurações de treinamento
            n_modelos_comparar=3,
            otimizar_hiperparametros=True,
            n_iter_otimizacao=10,
            salvar_modelo_final=True,
            nome_modelo="modelo_conforto_termico",
            pasta_modelos="../modelos"
        )
        
        print("\n" + "="*80)
        print("✅ PIPELINE DE TREINAMENTO CONCLUÍDO!")
        print("="*80)
        
        # Melhor modelo
        melhor_modelo_nome = str(resultado_treinamento['tabela_comparacao'].index[0])
        print(f"\n🏆 Melhor Modelo: {melhor_modelo_nome}")
        
        # Métricas
        print(f"\n📊 Métricas do Melhor Modelo:")
        metricas = resultado_treinamento['metricas_melhor']
        metricas_principais = ['MAE', 'MSE', 'RMSE', 'R2', 'MAPE']
        for nome in metricas_principais:
            if nome in metricas:
                valor = metricas[nome]
                if isinstance(valor, (int, float)):
                    print(f"  • {nome}: {valor:.4f}")
        
        # Model ID
        model_id = resultado_treinamento.get('model_id', 'N/A')
        if model_id and model_id != 'N/A':
            model_id_display = model_id
        else:
            model_id_display = "N/A (modo offline ou erro)"
        
        print(f"\n💾 Modelo:")
        print(f"  • Salvo em: {resultado_treinamento.get('caminho_modelo', 'N/A')}")
        print(f"  • ClearML ID: {model_id_display}")
        
        # Comparação de modelos
        print(f"\n📋 Comparação de Modelos (Top {len(resultado_treinamento['tabela_comparacao'])}):")
        print(resultado_treinamento['tabela_comparacao'][['MAE', 'MSE', 'RMSE', 'R2']].head())
        
        print(f"\n🌐 Acesse: https://app.clear.ml")
        print(f"  • Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS")
        print(f"  • Task: Pipeline_Treinamento")
        print(f"  • Modelo: modelo_{melhor_modelo_nome}_modelo_conforto_termico")
        print(f"  • Genealogia: dados_brutos → processados → features → **MODELO**")
        
        print("\n🎯 Modelo treinado disponível em:")
        print("  resultado_treinamento['melhor_modelo']")
        print("="*80)
        
    except Exception as e:
        print("\n" + "="*80)
        print("❌ ERRO DETECTADO")
        print("="*80)
        print(f"\nTipo do erro: {type(e).__name__}")
        print(f"Mensagem: {str(e)}")
        print("\n" + "-"*80)
        print("TRACEBACK COMPLETO:")
        print("-"*80)
        traceback.print_exc()
        print("\n" + "="*80)
        print("💡 Dicas:")
        print("  • Execute os Pipelines 1 e 2 primeiro")
        print("  • Verifique se o DataFrame tem a coluna alvo")
        print("  • Use offline_mode=True para executar sem ClearML")
        print("="*80)
else:
    print("\n" + "="*80)
    print("❌ ERRO: Nenhum dado disponível")
    print("="*80)
    print("\n💡 Execute os pipelines na ordem:")
    print("  1. Pipeline 1: Processamento")
    print("  2. Pipeline 2: Features")
    print("  3. Pipeline 3: Treinamento (esta célula)")
    print("="*80)

PIPELINE DE TREINAMENTO - CLEARML ONLINE
✓ Pipeline 2 concluído
  Dataset ID anterior: 80272a4490934b8ba191125924fc6c34
PIPELINE DE TREINAMENTO - CLEARML ONLINE
✓ Pipeline 2 concluído
  Dataset ID anterior: 80272a4490934b8ba191125924fc6c34


2026-02-09 22:53:49,513 - INFO - Modo ONLINE: Iniciando integração ClearML



Dados para treinamento: (1720, 78)
Coluna alvo: p1



2026-02-09 22:53:49,738 - INFO - ✓ Credenciais ClearML carregadas do .env
2026-02-09 22:53:49,739 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:53:49,739 - INFO -   API Host: https://api.clear.ml
2026-02-09 22:53:49,740 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:53:49,740 - INFO -   Web Host: https://app.clear.ml
2026-02-09 22:53:49,818 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-09 22:53:49,818 - INFO - ✓ ClearML configurado para uso ONLINE
2026-02-09 22:53:49,879 - INFO - Fechando task anterior: Pipeline_Treinamento
2026-02-09 22:53:49,879 - INFO - Fechando task anterior: Pipeline_Treinamento


ClearML Task: created new task id=563165a5492d44119a249e32223ada39
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/563165a5492d44119a249e32223ada39/output/log
ClearML results page: https://app.clear.ml/projects/06661800c2e642339d895b6d02c06481/experiments/563165a5492d44119a249e32223ada39/output/log


2026-02-09 22:54:09,465 - INFO - Task criada: Pipeline_Treinamento (ID: 563165a5492d44119a249e32223ada39, Projeto: Estudo de Sensação Térmica Humana em Santa Maria - RS)
